In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("saved_dataframes/complete_df.csv")
df = df.dropna()
df.head()

,content,year,month,day,about,subtitles
2156,hier ist das erste deutsche Fernsehen mit der ...,2013,4,21,Hinweis: Der Beitrag zur Fußball-Bundesliga da...,Diese Sendung wurde vom NDR live untertitelt (...
2157,hier ist das erste deutsche Fernsehen mit der ...,2013,4,22,"Themen der Sendung: Warnstreik Lufthansa, Mind...",Herzlich willkommen zur Live- Untertitelung de...
2158,hier ist das erste deutsche Fernsehen mit der ...,2013,4,23,"Themen der Sendung: Der Steuer-Fall Hoeneß, FC...",Diese Sendung wurde live untertitelt.(23.04.20...
2159,hier ist das erste deutsche Fernsehen mit der ...,2013,4,24,Themen der Sendung: Debatte über Steuerhinterz...,Die Sendung wurde vom NDR live untertitelt. (2...
2160,hier ist das erste deutsche Fernsehen mit der ...,2013,4,25,"Themen der Sendung: Bundestag zu Mindestlohn, ...",Diese Sendung wurde vom NDR live untertitelt (...


In [3]:
df = df[df["about"].str.contains("Themen")]
df.head()

,content,year,month,day,about,subtitles
2157,hier ist das erste deutsche Fernsehen mit der ...,2013,4,22,"Themen der Sendung: Warnstreik Lufthansa, Mind...",Herzlich willkommen zur Live- Untertitelung de...
2158,hier ist das erste deutsche Fernsehen mit der ...,2013,4,23,"Themen der Sendung: Der Steuer-Fall Hoeneß, FC...",Diese Sendung wurde live untertitelt.(23.04.20...
2159,hier ist das erste deutsche Fernsehen mit der ...,2013,4,24,Themen der Sendung: Debatte über Steuerhinterz...,Die Sendung wurde vom NDR live untertitelt. (2...
2160,hier ist das erste deutsche Fernsehen mit der ...,2013,4,25,"Themen der Sendung: Bundestag zu Mindestlohn, ...",Diese Sendung wurde vom NDR live untertitelt (...
2161,hier ist das erste deutsche Fernsehen mit der ...,2013,4,26,Themen der Sendung: Bundesparteitag der Grünen...,Herzlich willkommen zur Live- Untertitelung de...


In [4]:
df["about"] = df["about"].apply(lambda x: x.replace("Themen der Sendung: ", ""))
df.head()

,content,year,month,day,about,subtitles
2157,hier ist das erste deutsche Fernsehen mit der ...,2013,4,22,"Warnstreik Lufthansa, Mindestlohn für Friseure...",Herzlich willkommen zur Live- Untertitelung de...
2158,hier ist das erste deutsche Fernsehen mit der ...,2013,4,23,"Der Steuer-Fall Hoeneß, FC Bayern vor CL-Halbf...",Diese Sendung wurde live untertitelt.(23.04.20...
2159,hier ist das erste deutsche Fernsehen mit der ...,2013,4,24,"Debatte über Steuerhinterziehung, BVerfG-Urtei...",Die Sendung wurde vom NDR live untertitelt. (2...
2160,hier ist das erste deutsche Fernsehen mit der ...,2013,4,25,"Bundestag zu Mindestlohn, Wirtschaft missachte...",Diese Sendung wurde vom NDR live untertitelt (...
2161,hier ist das erste deutsche Fernsehen mit der ...,2013,4,26,"Bundesparteitag der Grünen, Von der Leyen zieh...",Herzlich willkommen zur Live- Untertitelung de...


In [5]:
about = df["about"].to_list()
about = [x.split(",") for x in about]
about = [item for sublist in about for item in sublist]

In [6]:
df_about = pd.DataFrame(about, columns=["content"])
df_about.head()

,content
0,Warnstreik Lufthansa
1,Mindestlohn für Friseure
2,Hoeneß-Affäre
3,Kritik an von der Leyen
4,Lafontaine tritt nicht zur Bundestagswahl an


In [7]:
df_about = pd.read_csv("saved_dataframes/about_preprocessed.csv")
df_about.head()

,content
0,"['warnstreik', 'lufthansa']"
1,"['mindestlohn', 'friseur']"
2,['hoeneß-affäre']
3,"['kritik', 'leyen']"
4,"['lafontaine', 'tritt', 'bundestagswahl']"


In [8]:
from gensim.models import Word2Vec
import gensim.models.keyedvectors as keyed
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
model_my = Word2Vec.load("w2v_models/complete_w2v.model")
df_about_my = df_about.dropna()

In [10]:
df_about_my = df_about_my.loc[df_about_my["content"] != "[]"]

In [11]:
import numpy as np
import ast

def get_similarities(df_about, model, selected=None):
    similarity_scores = []
    indices = []
    if selected is None:
        selected = list(df_about.index)
    i = 0
    for row in df_about.itertuples():
        if i in selected:
            content = ast.literal_eval(row.content)
            try:
                embed = model[content]
                #sim = np.mean(cosine_similarity(embed, embed))
                sim = cosine_similarity(embed, embed).ravel().tolist()
                similarity_scores.append(sim)
                indices.append(i)
            except KeyError:
                print("Model does not know: {}".format(content))
        i += 1
    return similarity_scores, indices

In [ ]:
sim_myembed, indices_myembed = get_similarities(df_about_my, model_my)

/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() ins

Model does not know: ['hoeneß-affäre']
Model does not know: ['kritik', 'leyen']
Model does not know: ['mutmaßlicher', 'boston-attentäter', 'verhören']
Model does not know: ['abkomme', 'kosovo', 'serbien']
Model does not know: ['prozess', 'ehemalig', 'griechisch', 'minister']
Model does not know: ['ausstellung', 'moskau', 'rekonstruieren', 'pergamonaltar']
Model does not know: ['wetter']
Model does not know: ['steuer-fall', 'hoeneß']
Model does not know: ['fc', 'bayer', 'cl-halbfinale']
Model does not know: ['frankreich', 'führen', 'homo-ehe']
Model does not know: ['terroranschlag', 'kanada', 'vereiteln']
Model does not know: ['boston-attentäter', 'verhören']
Model does not know: ['außenminister', 'nato', 'syrien']
Model does not know: ['attentat', 'frz', 'botschaft', 'libyen']
Model does not know: ['festnetz-datenlimit', 'telekom']
Model does not know: ['abiturientenproteste', 'nrw']
Model does not know: ['vivi', 'bach', 'sterben']
Model does not know: ['wetter']
Model does not know: [

Model does not know: ['sinkende', 'pegelstände', 'hochwasser']
Model does not know: ['wetter']
Model does not know: ['g8', 'finden', 'minimalkonsens', 'syrien']
Model does not know: ['afghanistan', 'übernehmen', 'sicherheitsverantwortung']
Model does not know: ['festnahme', 'türkei']
Model does not know: ['griechischer', 'staatsrundfunk', 'senden']
Model does not know: ['neues', 'verfahren', 'zschäpe']
Model does not know: ['ig-metall-befragung']
Model does not know: ['hochwasser', 'deutschland']
Model does not know: ['wetter']
Model does not know: ['streit', 'taliban-gespräche']
Model does not know: ['erneut', 'protest', 'brasilien']
Model does not know: ['anschlag', 'somalia']
Model does not know: ['protest', 'türkei']
Model does not know: ['brunsbüttel', 'verlieren', 'betriebserlaubnis']
Model does not know: ['commerzbank', 'bauen', 'stelle', 'ab']
Model does not know: ['einigung', 'fluthilfen']
Model does not know: ['überschwemmung', 'pyrenäen']
Model does not know: ['ungarischer',

/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() ins

Model does not know: ['folge', 'ägypten-unruhen']
Model does not know: ['neues', 'vorwurf', 'nsa']
Model does not know: ['ber-aufsichtsratssitzung']
Model does not know: ['bahn', 'werben', 'ex-mitarbeiter']
Model does not know: ['ärzte-einkommen', 'steigen']
Model does not know: ['bundesrat', 'geben', 'fluthilfe', 'frei']
Model does not know: ['erdbeben', 'neuseeland']
Model does not know: ['leichtathletik-wm']
Model does not know: ['wetter']
Model does not know: ['lage', 'ägypten']
Model does not know: ['150', 'jahr', 'spd']
Model does not know: ['ber-teileröffnung']
Model does not know: ['bahn-chef', 'entschuldigen']
Model does not know: ['flüchtlingsdemo', 'hamburg']
Model does not know: ['fährunglück', 'philippinen']
Model does not know: ['fußball-bundesliga']
Model does not know: ['leichtathletik-wm']
Model does not know: ['wetter']
Model does not know: ['machtkampf', 'ägypten']
Model does not know: ['koalitionsausage', 'wahlkampf']
Model does not know: ['seehofer', 'länderfinanza

...

/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() ins


Model does not know: ['wetter']
Model does not know: ['osterfeiern', 'jerusalem', 'bethlehem']
Model does not know: ['ostergottesdienste', 'deutschland']
Model does not know: ['politiker', 'fordern', 'mehr', 'einsatz', 'fremdenfeindlichkeit']
Model does not know: ['luftfahrtbundesamt', 'wusste', 'medizinische', 'vorgeschichte', 'germanwings-copiloten']
Model does not know: ['land', 'neue', 'beweis', 'npd-verbotsverfahren', 'vorlegen']
Model does not know: ['kampf', 'jemen', 'gehen', 'weit']
Model does not know: ['mehrtägige', 'staatstrauer', 'kenia']
Model does not know: ['italienisch', 'marine', 'retten', 'mindestens', '1500', 'bootsflüchtlinge', 'mittelmeer']
Model does not know: ['fußball-bundesliga']
Model does not know: ['handball-bundesliga']
Model does not know: ['wetter']
Model does not know: ['konflikt', 'jemen', 'spitzen', 'weit']
Model does not know: ['is', 'greifen', 'flüchtlingslager', 'damaskus']
Model does not know: ['türkei', 'sperren', 'zugang', 'youtube', 'twitter']


/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() ins

Model does not know: ['sc', 'paderborn', 'sc', 'freiburg', '1', 'fußball-bundesliga', 'absteigen']
Model does not know: ['lottozahlen']
Model does not know: ['wetter']
Model does not know: ['debatte', 'homo-ehe', 'deutschland']
Model does not know: ['spd', 'fordern', 'aufklärung', 'nsa-affäre']
Model does not know: ['anti-ngo-gesetz', 'russland']
Model does not know: ['stichwahl', 'präsidentenamt', 'pol']
Model does not know: ['protest', 'cleveland']
Model does not know: ['bundespolizei', 'führen', 'beschwerdestelle']
Model does not know: ['amazon', 'versteuern', 'gewinn', 'deutschland']
Model does not know: ['kardinal', 'marx', 'halten', 'pfingstpredigt']
Model does not know: ['saisonfinale', 'fußball-bundesliga']
Model does not know: ['formel', '1']
Model does not know: ['wetter']
Model does not know: ['andrzej', 'duda', 'präsident', 'pol']
Model does not know: ['griechenland', 'iwf-kredite', 'zurückzahlen']
Model does not know: ['schweiz', 'liste', 'steuersündern', 'veröffentlichen'

/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/timur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() ins

In [ ]:
model_other = keyed.KeyedVectors.load_word2vec_format('w2v_models/german.model', binary=True)
df_about_other = pd.read_csv("saved_dataframes/about_preprocessed_notlower.csv")
df_about_other = df_about_other.dropna()
df_about_other = df_about_other.loc[df_about["content"] != "[]"]
df_about_other.head()

In [ ]:
sim_other, indices_other = get_similarities(df_about_other, model_other)

In [ ]:
intersect = set(indices_myembed).intersection(set(indices_other))

In [ ]:
sim_myembed, indices_myembed = get_similarities(df_about_my, model_my, intersect)
sim_other, indices_other = get_similarities(df_about_other, model_other, intersect)

In [ ]:
print(np.mean([item for sublist in sim_myembed for item in sublist]))
print(np.mean([item for sublist in sim_other for item in sublist]))